<a href="https://colab.research.google.com/github/bhavya-kashyap/web-scrapping/blob/main/web_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [3]:
import json
import numpy as np
import pandas as pd         
import ctypes

In [4]:
hospitals = pd.read_csv('https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/GA_Hospitals.csv')
baseurl = 'https://freida.ama-assn.org/institution/'
URLS = []
for index, i in hospitals.iterrows():
  URLS.append(str(baseurl + str(i['H.No.'])))
hospitals['URLS'] = URLS

hospitals = hospitals.fillna('')
hospitals['Med_center'] = hospitals['Med_center'] + " " + hospitals['Unnamed: 2']
hospitals = hospitals.drop('Unnamed: 2', axis=1)


hospitals.head(5)

,H.No.,Med_center,URLS
0,120127,Abraham Infectious Disease Associates,https://freida.ama-assn.org/institution/120127
1,128070,AG Rhodes Homes,https://freida.ama-assn.org/institution/128070
2,129528,Air Methods Air Life Georgia,https://freida.ama-assn.org/institution/129528
3,129505,Albany Arthritis and Orthopedic Center,https://freida.ama-assn.org/institution/129505
4,120063,Alfred E Beach High School (APEX Program by G...,https://freida.ama-assn.org/institution/120063


In [5]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

def pagesource(url, driver):
    driver = driver
    driver.get(url)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source)
    #driver.close()
    return soup


In [ ]:
mc = []
add = []
d = driversetup()
for index, i in hospitals.iterrows():
  url = i['URLS']
  s = pagesource(url, d)
  med_school = list(s.findAll('small', class_="bold"))
  address = list(s.findAll('div', class_="details__address ng-star-inserted"))
  address[0].br.string = ', '
  print(address[0].text)
  if med_school == []:
    mc.append("")
    print("Medical School : ", mc[index])
  elif len(med_school) == 1:
    for j in range(len(med_school)):
      mc.append(med_school[j].text)
      print("Medical School : ", mc[index])
  elif len(med_school) > 1:
    list_med_schools = []
    for j in range(len(med_school)):
      list_med_schools.append(med_school[j].text)
    mc.append(list_med_schools)
    print("Medical School : ", mc[index])
    
  if address != []:
    #i['Address'] = address[0].text
    add.append(address[0].text)
    print("Address of hospital : ", add[index])
  else:
    add.append("")
    print("Address of hospital : ", add[index])


hospitals['Med_School'] = mc
hospitals['Address'] = add
hospitals.head(10)

In [11]:
#print(len(mc),len(add))
hospitals.head(20)

,H.No.,Med_center,URLS,Med_School,Address
0,120127,Abraham Infectious Disease Associates,https://freida.ama-assn.org/institution/120127,,"2024 E Pinetree Blvd, Thomasville, GA 31792"
1,128070,AG Rhodes Homes,https://freida.ama-assn.org/institution/128070,,"3715 Northside Pkwy, Atlanta, GA 30327"
2,129528,Air Methods Air Life Georgia,https://freida.ama-assn.org/institution/129528,,"635 Regional Airport Rd, Carrollton, GA 30117"
3,129505,Albany Arthritis and Orthopedic Center,https://freida.ama-assn.org/institution/129505,,"2100 Palmyra Rd, Albany, GA 31701"
4,120063,Alfred E Beach High School (APEX Program by G...,https://freida.ama-assn.org/institution/120063,,"3001 Hopkins St, Savannah, GA 31405"
5,128064,Ambulatory Care Center Medical College of Geo...,https://freida.ama-assn.org/institution/128064,,"1120 15th St, Augusta, GA 30912"
6,128103,American Cancer Society Georgia,https://freida.ama-assn.org/institution/128103,Morehouse Sch of Med,"250 Williams St NW, Atlanta, GA 30303"
7,120130,Archbold Gastroenterology Group,https://freida.ama-assn.org/institution/120130,,"112 Mimosa Dr, Thomasville, GA 31792"
8,128069,Archbold Medical Center,https://freida.ama-assn.org/institution/128069,,"915 Gordon Ave, Thomasville, GA 31799"
9,120128,Archbold Pulmonary Medicine and Critical Care...,https://freida.ama-assn.org/institution/120128,,"2705 E Pinetree Blvd, Thomasville, GA 31792"
